[View in Colaboratory](https://colab.research.google.com/github/fayzii/Scrap_NationalWeatherService/blob/master/Scrap_NationalWeatherService.ipynb)

In [0]:
import requests
from bs4 import BeautifulSoup

In [2]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Mostly cloudy, with a low around 54. West southwest wind around 11 mph. " class="forecast-icon" src="newimages/medium/nbkn.png" title="Overnight: Mostly cloudy, with a low around 54. West southwest wind around 11 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 54 °F
 </p>
</div>


In [3]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Overnight
Mostly Cloudy
Low: 54 °F


In [4]:
img = tonight.find("img")
desc = img['title']

print(desc)

Overnight: Mostly cloudy, with a low around 54. West southwest wind around 11 mph. 


In [5]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Overnight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight']

In [6]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Mostly Cloudy', 'Mostly Cloudythen Sunnyand Breezy', 'IncreasingClouds andBreezy thenMostly Cloudy', 'Partly Sunnyand Breezy', 'Partly Cloudyand Breezythen MostlyCloudy', 'Mostly Sunny', 'Mostly Cloudy', 'Mostly Sunny', 'Mostly Cloudy']
['Low: 54 °F', 'High: 63 °F', 'Low: 54 °F', 'High: 63 °F', 'Low: 54 °F', 'High: 65 °F', 'Low: 55 °F', 'High: 67 °F', 'Low: 54 °F']
['Overnight: Mostly cloudy, with a low around 54. West southwest wind around 11 mph. ', 'Wednesday: Cloudy through mid morning, then gradual clearing, with a high near 63. Breezy, with a west southwest wind 11 to 16 mph increasing to 17 to 22 mph in the afternoon. Winds could gust as high as 28 mph. ', 'Wednesday Night: Increasing clouds, with a low around 54. Breezy, with a west southwest wind 16 to 22 mph, with gusts as high as 29 mph. ', 'Thursday: Partly sunny, with a high near 63. Breezy, with a west southwest wind 16 to 24 mph, with gusts as high as 32 mph. ', 'Thursday Night: Mostly cloudy, with a low around 54. Bre

In [7]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,desc,period,short_desc,temp
0,"Overnight: Mostly cloudy, with a low around 54...",Overnight,Mostly Cloudy,Low: 54 °F
1,"Wednesday: Cloudy through mid morning, then gr...",Wednesday,Mostly Cloudythen Sunnyand Breezy,High: 63 °F
2,"Wednesday Night: Increasing clouds, with a low...",WednesdayNight,IncreasingClouds andBreezy thenMostly Cloudy,Low: 54 °F
3,"Thursday: Partly sunny, with a high near 63. B...",Thursday,Partly Sunnyand Breezy,High: 63 °F
4,"Thursday Night: Mostly cloudy, with a low arou...",ThursdayNight,Partly Cloudyand Breezythen MostlyCloudy,Low: 54 °F
5,"Friday: Mostly sunny, with a high near 65.",Friday,Mostly Sunny,High: 65 °F
6,"Friday Night: Mostly cloudy, with a low around...",FridayNight,Mostly Cloudy,Low: 55 °F
7,"Saturday: Mostly sunny, with a high near 67.",Saturday,Mostly Sunny,High: 67 °F
8,"Saturday Night: Mostly cloudy, with a low arou...",SaturdayNight,Mostly Cloudy,Low: 54 °F


In [8]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    54
1    63
2    54
3    63
4    54
5    65
6    55
7    67
8    54
Name: temp_num, dtype: object

In [9]:
weather["temp_num"].mean()

58.77777777777778

In [10]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [11]:
weather[is_night]

,desc,period,short_desc,temp,temp_num,is_night
0,"Overnight: Mostly cloudy, with a low around 54...",Overnight,Mostly Cloudy,Low: 54 °F,54,True
2,"Wednesday Night: Increasing clouds, with a low...",WednesdayNight,IncreasingClouds andBreezy thenMostly Cloudy,Low: 54 °F,54,True
4,"Thursday Night: Mostly cloudy, with a low arou...",ThursdayNight,Partly Cloudyand Breezythen MostlyCloudy,Low: 54 °F,54,True
6,"Friday Night: Mostly cloudy, with a low around...",FridayNight,Mostly Cloudy,Low: 55 °F,55,True
8,"Saturday Night: Mostly cloudy, with a low arou...",SaturdayNight,Mostly Cloudy,Low: 54 °F,54,True
